# Import the Library

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, MinMaxScaler

#  Loading the Data As a DataFrame

In [2]:
animes = pd.read_feather("../input/anime-data-save-as-feather/animes.feather")
rating_df = pd.read_feather("../input/anime-data-save-as-feather/rating_df.feather")

# PreProcessing

공통

In [3]:
""" rating 데이터셋"""

n_ratings = rating_df['user_id'].value_counts()

''' 5000개 이상 평가한 유저로 필터'''
rating_df = rating_df[rating_df['user_id'].isin(n_ratings[n_ratings >= 5000].index)].copy()


""" 결측치 처리 """

animes = animes.dropna()
rating_df = rating_df.dropna()

""" 중복값 처리 """

def remove_duplicated_rows(df):
    print('dataframe')
    duplicates = df.duplicated()
    if duplicates.sum() > 0:
        print('> {} duplicates'.format(duplicates.sum()))
        df = df[~duplicates]
    print('> {} duplicates'.format(df.duplicated().sum()))

remove_duplicated_rows(animes)
remove_duplicated_rows(rating_df)

dataframe
> 0 duplicates
dataframe
> 0 duplicates


## anime data

## 행 확인 및 삭제

In [4]:
animes.rename(columns = {'Name_x': 'Name','Score_x': 'Score', 'Genders_x' : 'Genders'}, inplace = True)

In [5]:
animes

,MAL_ID,Name,Score,Genders,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0,"other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0,ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0,It is the dark century and the people are suff...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",Unknown,盗墓笔记之秦岭神树,ONA,Unknown,"Apr 4, 2021 to ?",Unknown,...,Unknown,Unknown,1.0,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,Unknown,2021 to ?,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,Unknown,"Jul, 2021 to ?",Summer 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",Unknown,ヤマノススメ Next Summit,TV,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,New Yama no Susume anime.


### 여기서 Column 설명

* MAL_ID:  애니 id 
* Name: 애니 이름
* Score: 이 만화가 전체 유저에게 평점을 얼마나 받았느냐
* Genders: 이 만화는 어디 장르에 속하냐, 혹은 무슨 장르를 지니고 있느냐(그래서 genders 라고 명명된것같긴한데 오해가 일어나니 genres로 바꿈)
* English name: 애니 영어이름
* Japanese name: 애니 일본어 이름
* Type: 애니 타입.. tv나 music movie ova 등
* Episodes': 총 몇 챕터로 이루어져있냐
* Aired: 방영 시작, 종영 날짜
* Premiered: season premiere. (e.g. Spring 1998)
* Producers: 만든사람 누구냐
* Licensors: 저작권 어디 있냐
* Studios: 어디 스튜디오에서 만들었느냐
* Source: 만화냐 소설이냐 책이냐...등
* Duration: 각 에피소드당 길이
* Rating: 만화의 등급
* Ranked: score 기준 몇등
* Popularity: 유저 목록에 만화가 추가되어있는 횟수
* Members: 만화에 해당하는 커뮤니티에 속해 있는 유저수
* Favorites: 이 만화를 좋아요 누른 사람 수
* Watching: 이 만화를 보고있는 중인 사람 수
* Completed: 이 만화를 다 본 사람 수
* On-Hold: 이 만화를 보다가 보류중인 사람 수
* Dropped: 이 만화를 보다가 버린 사람 수
* Plan to Watch': 이 만화를 볼 계획인 사람 수 
* Score-10': number of users who scored 10. (e.g. 229170)
* Score-9': number of users who scored 9. (e.g. 182126)
* Score-8': number of users who scored 8. (e.g. 131625)
* Score-7': number of users who scored 7. (e.g. 62330)
* Score-6': number of users who scored 6. (e.g. 20688)
* Score-5': number of users who scored 5. (e.g. 8904)
* Score-4': number of users who scored 4. (e.g. 3184)
* Score-3': number of users who scored 3. (e.g. 1357)
* Score-2': number of users who scored 2. (e.g. 741)
* Score-1': number of users who scored 1. (e.g. 1580)
* sypnopsis : 해당 만화의 설명

1. 남길 것들

    * MAL_ID: 애니 id - >anime_id 로 이름 바꿈, rating df와 이름 통일
    
    * Name: 애니 이름
    
    * Ranked: score 기준 몇등
    
    * Genders: 해당 만화는 어디 장르에 속하냐, 혹은 무슨 장르를 지니고 있느냐(그래서 genders 라고 명명된것같긴한데 오해가 일어나니 genres로 바꿈)
    
    > 이중에 remove_necessary = ['Hentai', 'Ecchi', 'Harem', 'Shoujo Ai', 'Shounen Ai', 'Yaoi'] 얘들은 날림
    
    * Rating: 해당 만화의 등급
    
    * Source: 해당 만화냐 소설이냐 책이냐...등
    
    * Duration: 각 에피소드당 길이
    
    * Episodes': 총 몇 챕터로 이루어져있냐
    
    * Type: 해당 애니 타입.. tv나 music movie ova 등
    
    * Aired: 해당 애니 방영 시작, 종영 날짜
    
    * N_votes : score-#을 1개로 통일, 몇명이 투표했냐.
    
    * sypnopsis : 해당 애니의 설명
    
    * 평가 신뢰도(가칭): watching, completed, on-hold, dropped 들을 1개로 만들기.
    
    * 기대 신뢰도(가칭): popularity, members, favorites, plan to watch 들을 1개로 만들기


2. 날릴 것들

1) name으로 대체


    * English name: 애니 영어이름
    * Japanese name: 애니 일본어 이름

2) unknown이 너무 많음


    * Producers: 만든사람 누구냐
    * Licensors: 저작권 어디 있냐
    * Studios: 어디 스튜디오에서 만들었느냐

3) score에 결측치가 너무 많음

    * Score: 해당 만화가 유저들에게 받은 평점의 평균

    * Premiered: season premiere. (e.g. Spring 1998) 

4) 평가 신뢰도(가칭) 으로 대체


    * Watching: 이 만화를 보고있는 중인 사람 수
    * Completed: 이 만화를 다 본 사람 수
    * On-Hold: 이 만화를 보다가 보류중인 사람 수
    * Dropped: 이 만화를 보다가 버린 사람 수

5) 기대 신뢰도(가칭)으로 대체


    * Popularity: 유저 목록에 만화가 추가되어있는 횟수
    * Members: 만화에 해당하는 커뮤니티에 속해 있는 유저수
    * Favorites: 이 만화를 좋아요 누른 사람 수
    * Plan to Watch': 이 만화를 볼 계획인 사람 수

6) 총 평가수로 대체


    * Score-10': number of users who scored 10. (e.g. 229170)
    
    * Score-9': number of users who scored 9. (e.g. 182126)
    
    * Score-8': number of users who scored 8. (e.g. 131625)
    
    * Score-7': number of users who scored 7. (e.g. 62330)
    
    * Score-6': number of users who scored 6. (e.g. 20688)
    
    * Score-5': number of users who scored 5. (e.g. 8904)
    
    * Score-4': number of users who scored 4. (e.g. 3184)
    
    * Score-3': number of users who scored 3. (e.g. 1357)
    
    * Score-2': number of users who scored 2. (e.g. 741)
    
    * Score-1': number of users who scored 1. (e.g. 1580)

#### column명 변경

In [6]:
animes.rename(columns = {'MAL_ID': 'anime_id','Genders' : 'Genres'}, inplace = True)

#### 필요없는 column 삭제

In [7]:
animes = animes.drop(['Japanese name','English name','Premiered', 'Producers','Licensors', 'Studios',],axis =1)

#### 장르 날림

In [8]:
remove_necessary = ['Hentai', 'Ecchi', 'Harem', 'Shoujo Ai', 'Shounen Ai', 'Yaoi']
to_drop=[]
for i in remove_necessary:
    for index, row in animes.iterrows():
        if i in row['Genres']:
            to_drop.append(row['anime_id'])
            animes.drop(index,inplace=True)

### column 삭제 및 1개로 만들기 

#### vote 수 column 생성 및 score-# column 삭제

In [9]:
def get_voters(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10):
    x = [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10]
    def f(val) :
        if val == "Unknown" : return 0
        return float(val)
    return int(sum(map(f, x)))
animes["N_votes"] = animes.apply(lambda x: get_voters(x['Score-1'], x['Score-2'], x['Score-3'], 
                                                      x['Score-4'], x['Score-5'], x['Score-6'],
                                                      x['Score-7'], x['Score-8'], x['Score-9'], 
                                                      x['Score-10']), axis=1)

In [10]:
animes = animes.drop(['Score-10','Score-9','Score-8',
                      'Score-7','Score-6','Score-5','Score-4',
                      'Score-3','Score-2','Score-1'],axis =1)

#### 평가 신뢰도(가칭) 만들기

In [11]:
animes['rating_reliability'] = animes.apply(lambda x: float(x['Completed']) * 0.4 
                                            + float(x['Watching']) * 0.25 
                                            + float(x['On-Hold']) * 0.25 
                                            + float(x['Dropped']) * 0.1,axis='columns')

In [12]:
animes[["rating_reliability"]] = MinMaxScaler().fit_transform(animes[["rating_reliability"]])

In [13]:
animes = animes.drop(['Completed','Watching','On-Hold','Dropped'],axis =1)

#### 기대 신뢰도(가칭) 만들기

In [14]:
animes['expected_reliability'] = animes.apply(lambda x: float(x.Popularity) * 0.4 
                                              + float(x.Members) * 0.25 
                                             + float(x.Favorites) * 0.25 
                                              + float(x['Plan to Watch']) * 0.1,axis='columns')

In [15]:
animes[["expected_reliability"]] = MinMaxScaler().fit_transform(animes[["expected_reliability"]])

In [16]:
animes = animes.drop(['Popularity','Members','Favorites','Plan to Watch'],axis =1)

music 데이터 날리기

In [17]:
for index, row in animes.iterrows():
    if row.Type == 'Music':
        to_drop.append(row['anime_id'])
        animes.drop(index,inplace=True)

#### anime data 전처리 결과

In [18]:
animes

,anime_id,Name,Score,Genres,Type,Episodes,Aired,Source,Duration,Rating,Ranked,sypnopsis,N_votes,rating_reliability,expected_reliability
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,"Apr 3, 1998 to Apr 24, 1999",Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,"In the year 2071, humanity has colonized sever...",641705,3.625317e-01,0.516247
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,"Sep 1, 2001",Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,"other day, another bounty—such is the life of ...",160349,9.211236e-02,0.106516
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,"Apr 1, 1998 to Sep 30, 1998",Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,"Vash the Stampede is the man with a $$60,000,0...",286146,1.653254e-01,0.225272
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",TV,26,"Jul 2, 2002 to Dec 24, 2002",Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,ches are individuals with special powers like ...,39094,2.316542e-02,0.039668
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",TV,52,"Sep 30, 2004 to Sep 29, 2005",Manga,23 min. per ep.,PG - Children,3710.0,It is the dark century and the people are suff...,5923,3.674963e-03,0.007708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",ONA,Unknown,"Apr 4, 2021 to ?",Novel,Unknown,Unknown,Unknown,No synopsis information has been added to this...,1,0.000000e+00,0.007671
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",TV,Unknown,2021 to ?,Manga,Unknown,Unknown,Unknown,ko is a typical high school student whose life...,0,7.592003e-06,0.013557
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",TV,Unknown,"Jul, 2021 to ?",Visual novel,Unknown,R - 17+ (violence & profanity),Unknown,Sequel to Higurashi no Naku Koro ni Gou .,1,8.134289e-07,0.015699
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",TV,Unknown,Unknown,Manga,Unknown,PG-13 - Teens 13 or older,Unknown,New Yama no Susume anime.,0,5.043259e-06,0.010725


## Rating df

In [19]:
to_drop = list(set(to_drop))
len(to_drop)

2604

In [20]:
to_drop_int = list(map(lambda x: np.int64(x),to_drop))

In [21]:
rating_df = rating_df[~rating_df['anime_id'].isin(to_drop_int)]

In [22]:
rating_df = rating_df.reset_index()
rating_df = rating_df.drop(['index'],axis=1)
rating_df

,user_id,anime_id,rating
0,4773,40679,0
1,4773,44040,0
2,4773,42514,0
3,4773,40930,0
4,4773,40964,0
...,...,...,...
1288940,352811,8985,0
1288941,352811,5454,0
1288942,352811,16241,0
1288943,352811,18179,0


# 여기서 부터 영빈이 preprocessing 사용

### 1-1-1. 결측치('Unknown') 처리
- 사용 feature: 모두
    - sypnopsis: "No synopsis information ..." & "No synopsis has ..." 삭제
    - Source: 최빈값 대치
    - 나머지: 삭제

In [23]:
""" 결측치 갯수 """
animes[animes == 'Unknown'].count()

anime_id                   0
Name                       0
Score                   4324
Genres                    63
Type                      32
Episodes                 444
Aired                    286
Source                  3084
Duration                 511
Rating                   658
Ranked                   356
sypnopsis                  0
N_votes                    0
rating_reliability         0
expected_reliability       0
dtype: int64

In [24]:
"""
1) sypnopsis 
"""     
animes = animes[~animes['sypnopsis'].str.contains("No synopsis")]

In [25]:
"""
2) 나머지
"""
animes = animes[animes['Genres'] != 'Unknown']
animes = animes[animes['Episodes'] != 'Unknown']
animes = animes[animes['Aired'] != 'Unknown']
animes = animes[animes['Duration'] != 'Unknown']
animes = animes[animes['Type'] != 'Unknown']
# animes = animes[animes['Source'] != 'Unknown']
animes = animes[animes['Rating'] != 'Unknown']
animes = animes[animes['Ranked'] != 'Unknown']
print('remaining: ', animes.shape)

remaining:  (11858, 15)


In [26]:
""" 결측치 갯수 """
animes[animes == 'Unknown'].count()

anime_id                   0
Name                       0
Score                   2922
Genres                     0
Type                       0
Episodes                   0
Aired                      0
Source                  2596
Duration                   0
Rating                     0
Ranked                     0
sypnopsis                  0
N_votes                    0
rating_reliability         0
expected_reliability       0
dtype: int64

In [27]:
"""
reset index
"""
animes = animes.reset_index() 

### 1-1-2. 전처리 (numeric)
- 사용 feature: Aired, Duration, Ranked

In [28]:
""" 1) Aired: 날짜 형식 통일 (e.g., Spe 1, 2001 => 2001)  """

years  = []
months = []
for val in animes['Aired']:
    vr = val.split()
    y = 'Unknown'
    m = 'Unknown'
    for v in vr:
        if v.isdigit() and len(v) == 4 :
            y = v
            break
    for v in vr:
        if not v.isdigit() and len(v) >= 3 and v[0].isupper() and v != 'Unknown' :
            m = v[:3]
            break
        
    years += [ y ]
    months += [ m ]

animes['Year'] = years
animes['Month'] = months

month_to_number = {
'Jan' : 1,         
'Feb' : 2,         
'Mar' : 3,           
'Apr' : 4,              
'May' : 5, 
'Jun' : 6,
'Jul' : 7, 
'Aug' : 8, 
'Sep' : 9, 
'Oct' : 10, 
'Nov' : 11, 
'Dec' : 12}

animes['Month'] = animes['Month'].replace(month_to_number)

animes['Year'] = animes['Year'].replace("Unknown", 0).astype(float)
animes['Month'] = animes['Month'].replace("Unknown", 0).astype(float)

animes['date'] = animes['Year'] + (animes['Month']/10)

In [29]:
""" 2) Duration: 단위 통일 (e.g., 1 hr. 55 min. => 115)  """

hrs  = []
mins = []
for val in animes['Duration']:
    split_list = val.split() # ['24', 'min.', 'per', 'ep.']
    h = 'Unknown'
    m = 'Unknown'
    for i in split_list:
        if i == 'hr.':
            h = split_list[split_list.index(i)-1]
        elif i == 'min.':
            m = split_list[split_list.index(i)-1]
        
    hrs += [ h ]
    mins += [ m ]

animes['hours'] = hrs
animes['mins'] = mins

animes['hours'] = animes['hours'].replace("Unknown", 0).astype(float)
animes['mins'] = animes['mins'].replace("Unknown", 0).astype(float)

animes['duration'] = (animes['hours']*60) + animes['mins']

In [30]:
""" 
3) Ranked: str -> int 타입 변환 
"""

try:
    animes['Ranked'] = animes['Ranked'].replace("Unknown", 0).astype(float)
except:
    pass
animes["Ranked"] = pd.to_numeric(animes["Ranked"])


In [31]:
animes = animes.drop(['Aired', 'Duration','Year', 'Month','hours', 'mins','index' , 'Score'], axis = 1)
animes.head()

,anime_id,Name,Genres,Type,Episodes,Source,Rating,Ranked,sypnopsis,N_votes,rating_reliability,expected_reliability,date,duration
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Original,R - 17+ (violence & profanity),28.0,"In the year 2071, humanity has colonized sever...",641705,0.362532,0.516247,1998.4,24.0
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Original,R - 17+ (violence & profanity),159.0,"other day, another bounty—such is the life of ...",160349,0.092112,0.106516,2001.9,115.0
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Manga,PG-13 - Teens 13 or older,266.0,"Vash the Stampede is the man with a $$60,000,0...",286146,0.165325,0.225272,1998.4,24.0
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",TV,26,Original,PG-13 - Teens 13 or older,2481.0,ches are individuals with special powers like ...,39094,0.023165,0.039668,2002.7,25.0
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",TV,52,Manga,PG - Children,3710.0,It is the dark century and the people are suff...,5923,0.003675,0.007708,2004.9,23.0


#### Source unknown 채우기

In [32]:
"""
4) Source
type이랑 duration 써서 끼워넣기
"""

'\n4) Source\ntype이랑 duration 써서 끼워넣기\n'

In [33]:
!pip install git+https://github.com/remykarem/mixed-naive-bayes#egg=mixed-naive-bayes

  Cloning https://github.com/remykarem/mixed-naive-bayes to /tmp/pip-install-ruf2xwxf/mixed-naive-bayes_e300d67d784f49c5942a01f229c99b9d
  Running command git clone -q https://github.com/remykarem/mixed-naive-bayes /tmp/pip-install-ruf2xwxf/mixed-naive-bayes_e300d67d784f49c5942a01f229c99b9d
  Created wheel for mixed-naive-bayes: filename=mixed_naive_bayes-0.0.2-py3-none-any.whl size=11223 sha256=3b84d0ad57250dc85b43e0e5f0ed935ab0db50eb82b7e360197ecde3107a3997
  Stored in directory: /tmp/pip-ephem-wheel-cache-g0hop9rm/wheels/f6/17/45/08ff7102e1201fe077c968291143479ecc63186e638bcce9f3
Successfully built mixed-naive-bayes


In [34]:
from mixed_naive_bayes import MixedNB
from sklearn.preprocessing import LabelEncoder

In [35]:
X = animes[animes['Source']  != 'Unknown'][{'Type','duration'}]
y = animes[animes['Source']  != 'Unknown'].Source

In [36]:
le1 = LabelEncoder()
le2 = LabelEncoder()

In [37]:
X['Type'] = le1.fit_transform(X['Type'])
y = le2.fit_transform(y)

In [38]:
clf = MixedNB(categorical_features=[1])

In [39]:
clf.fit(X,y)

MixedNB(alpha=0.5, var_smoothing=1e-09)

In [40]:
X_want = animes[animes['Source']  == 'Unknown'][{'Type','duration'}]
X_want['Type'] = le1.fit_transform(X_want['Type'])
X_want = X_want.reset_index() 

In [41]:
y_want = clf.predict(X_want[{'Type','duration'}])

In [42]:
X_want['Type'] = le1.inverse_transform(X_want['Type'])

In [43]:
y_want = le2.inverse_transform(y_want)

In [44]:
want_y = pd.DataFrame(y_want)

In [45]:
result_NB = pd.concat([X_want, want_y],axis=1)
result_NB.columns = ['anime_id','Type','duration','Source']

In [46]:
for i in range(len(result_NB)):
    animes.at[animes.index == result_NB.anime_id[i],'Source'] = result_NB.Source[i]

### 1-1-3. 전처리 (text) 
- 사용 feature: Genres, Sypnopsis, Type, Source, Rating
    - BoW: Genres
    - TF-IDF: sypnopsis
    - OneHotEncoding: Type, Source, Rating

In [47]:
""" 
1) BoW: Genres 
"""
#preprocess for bow(ex sci-fi -> scifi)
animes['Genres'] = animes['Genres'].str.replace(pat=r'-', repl=r'')
animes['Genres'] = animes['Genres'].str.replace(pat=r' ', repl=r'')

from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()
count_vector.fit(animes['Genres'])
#print('number of genres: ', len(count_vector.get_feature_names())) # action, comedy, ...

doc_array = count_vector.transform(animes['Genres']).toarray()
frequency_matrix = pd.DataFrame(doc_array, 
                                columns = count_vector.get_feature_names())
print('BoW of Genres shape: ', frequency_matrix.shape)

BoW of Genres shape:  (11858, 36)


In [48]:
""" 
2) tf-idf: sypnopsis 
"""

#animes['sypnopsis'] = animes['sypnopsis'].map(lambda x: ','.join(x))

def process_multilabel(series):
    series = series.split(" ")
    if "Unknown" in series:
        series.remove("Unknown")
    return series
animes["sypnopsis"] = animes["sypnopsis"].map(process_multilabel)

from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=100, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')
sypnopsis_original = animes['sypnopsis'].fillna('').astype(str) # Filling NaNs with empty string
sypnopsis_vector_tf_idf = tfv.fit_transform(sypnopsis_original)
sypnopsis_matrix = pd.DataFrame(sypnopsis_vector_tf_idf.toarray())
print("tf-idf of sypnopsis shape:", sypnopsis_vector_tf_idf.shape)

tf-idf of sypnopsis shape: (11858, 100)


In [49]:
""" 
3) OneHotEncoding: Type, Source, Rating 
"""

from sklearn.preprocessing import OneHotEncoder

Type = animes['Type'].values.reshape(-1, 1)
enc = OneHotEncoder()
enc.fit(Type)
type_ohe = enc.transform(Type).toarray()
type_ohe = pd.DataFrame(type_ohe, columns = enc.get_feature_names())

Source = animes['Source'].values.reshape(-1, 1)
enc = OneHotEncoder()
enc.fit(Source)
source_ohe = enc.transform(Source).toarray()
source_ohe = pd.DataFrame(source_ohe, columns = enc.get_feature_names())

Rating = animes['Rating'].values.reshape(-1, 1)
enc = OneHotEncoder()
enc.fit(Rating)
rating_ohe = enc.transform(Rating).toarray()
rating_ohe = pd.DataFrame(rating_ohe, columns = enc.get_feature_names())


## 1-2. score matrix (score_df)

In [50]:
a = len(rating_df.anime_id.unique())
b = len(rating_df.user_id.unique())
print(a)
print(b)

14949
185


In [51]:
print('animes: ',a)
print('users: ', b)
print('\n', '{0}명의 유저가 {1}개의 영화를 평가한 matrix 생성'.format(b,a), '\n')

import numpy as np

top_users = rating_df.groupby('user_id')['rating'].count()
top_r = rating_df.join(top_users, rsuffix='_r', how='inner', on='user_id')

top_animes = rating_df.groupby('anime_id')['rating'].count()
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='anime_id')

score_df = pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc=np.sum)
score_df

animes:  14949
users:  185

 185명의 유저가 14949개의 영화를 평가한 matrix 생성 



anime_id,1,5,6,7,8,15,16,17,18,19,...,48425,48426,48438,48441,48442,48456,48471,48483,48488,48491
user_id,,,,,,,,,,,,,,,,,,,,,
4773,8.0,8.0,8.0,7.0,6.0,NaN,8.0,7.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6852,6.0,4.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10255,10.0,10.0,8.0,8.0,8.0,8.0,10.0,9.0,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11100,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16057,10.0,0.0,0.0,NaN,NaN,0.0,0.0,10.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341571,9.0,9.0,8.0,NaN,NaN,8.0,9.0,NaN,9.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341719,7.0,7.0,5.0,7.0,0.0,8.0,0.0,7.0,8.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345498,9.0,9.0,9.0,6.0,NaN,5.0,9.0,5.0,9.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## For XGB DATAset

In [52]:
animes

,anime_id,Name,Genres,Type,Episodes,Source,Rating,Ranked,sypnopsis,N_votes,rating_reliability,expected_reliability,date,duration
0,1,Cowboy Bebop,"Action,Adventure,Comedy,Drama,SciFi,Space",TV,26,Original,R - 17+ (violence & profanity),28.0,"[In, the, year, 2071,, humanity, has, colonize...",641705,0.362532,0.516247,1998.4,24.0
1,5,Cowboy Bebop: Tengoku no Tobira,"Action,Drama,Mystery,SciFi,Space",Movie,1,Original,R - 17+ (violence & profanity),159.0,"[other, day,, another, bounty—such, is, the, l...",160349,0.092112,0.106516,2001.9,115.0
2,6,Trigun,"Action,SciFi,Adventure,Comedy,Drama,Shounen",TV,26,Manga,PG-13 - Teens 13 or older,266.0,"[Vash, the, Stampede, is, the, man, with, a, $...",286146,0.165325,0.225272,1998.4,24.0
3,7,Witch Hunter Robin,"Action,Mystery,Police,Supernatural,Drama,Magic",TV,26,Original,PG-13 - Teens 13 or older,2481.0,"[ches, are, individuals, with, special, powers...",39094,0.023165,0.039668,2002.7,25.0
4,8,Bouken Ou Beet,"Adventure,Fantasy,Shounen,Supernatural",TV,52,Manga,PG - Children,3710.0,"[It, is, the, dark, century, and, the, people,...",5923,0.003675,0.007708,2004.9,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11853,47616,Yakusoku no Neverland 2nd Season: Michishirube,"Mystery,Psychological,Supernatural,Thriller,Sh...",Special,1,Manga,R - 17+ (violence & profanity),10760.0,"[cap, of, the, first, season, of, Yakusoku, no...",6156,0.003931,0.007692,2021.2,23.0
11854,48177,Ichiban Chikakute Tooi Hoshi,"Music,Drama",ONA,1,Original,PG - Children,14582.0,"[one-minute, concept, animation, that, exists,...",37,0.000036,0.009104,2021.2,1.0
11855,48369,Wo Shi Da Shenxian 2nd Season,"Action,MartialArts,Fantasy,Shounen",ONA,16,Manga,PG-13 - Teens 13 or older,12882.0,"[Second, season, of, Wo, Shi, Da, Shen, Shan, .]",11,0.000013,0.009638,2021.2,20.0
11856,48374,Fei Ren Zai 4th Season,"SliceofLife,Comedy,Supernatural",ONA,12,Manga,PG-13 - Teens 13 or older,13980.0,"[Fourth, season, of, Fei, Ren, Zai, .]",6,0.000007,0.009724,2021.2,5.0


In [53]:
temp = pd.concat([type_ohe,source_ohe,rating_ohe,frequency_matrix,sypnopsis_matrix],axis=1)

In [54]:
animes_xgb = animes.drop(['Name','Genres','Type','Source','sypnopsis','Rating'], axis = 1)
animes_xgb = pd.concat([animes_xgb,temp],axis=1)
# animes_xgb = animes_xgb.set_index('anime_id')

In [55]:
animes_xgb

,anime_id,Episodes,Ranked,N_votes,rating_reliability,expected_reliability,date,duration,x0_Movie,x0_ONA,...,90,91,92,93,94,95,96,97,98,99
0,1,26,28.0,641705,0.362532,0.516247,1998.4,24.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.219417,0.000000,0.000000
1,5,1,159.0,160349,0.092112,0.106516,2001.9,115.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,6,26,266.0,286146,0.165325,0.225272,1998.4,24.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,7,26,2481.0,39094,0.023165,0.039668,2002.7,25.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.294035,0.000000,0.250298,0.000000,0.000000
4,8,52,3710.0,5923,0.003675,0.007708,2004.9,23.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.228532,0.438215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11853,47616,1,10760.0,6156,0.003931,0.007692,2021.2,23.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
11854,48177,1,14582.0,37,0.000036,0.009104,2021.2,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.580587,0.000000,0.000000,0.000000
11855,48369,16,12882.0,11,0.000013,0.009638,2021.2,20.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
11856,48374,12,13980.0,6,0.000007,0.009724,2021.2,5.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


## 1-3. user matrix (user_df)
- ani_df + score_df

In [56]:
all_users_df = []

user_id_list = list(score_df.index)

for user_id in user_id_list:
    """ 유저 별 score vector """
    score_vector = score_df.loc[user_id] 
    score_vector = score_vector.dropna() 
    score_vector = score_vector[score_vector != 0] # 0점은 평가 안한것 (여기서 많이 날라가는듯)

    anime_id_list = list(score_vector.index)
    user_score_list = list(score_vector.values)

    """ 유저가 본 영화만 추출 """
    animes_user = animes.loc[animes['anime_id'].isin(anime_id_list)] 

    """ 유저 별 매트릭스 생성 """
    score_vector_df = pd.DataFrame(score_vector)
    score_vector_df['anime__id'] = score_vector_df.index
    score_vector_df.columns = ['score_by_user_{}'.format(user_id), 'anime__id']

    user_df = pd.merge(animes_user,score_vector_df, how='inner',on='anime_id')
    user_df = user_df.iloc[: , :-1]

    """ append user_df """
    all_users_df.append(user_df)

a = len(all_users_df)
print('users: ', a)
print('\n', '{0}명의 유저에 대해 아래 matrix를 각각 생성'.format(a), '\n')
all_users_df[0]

users:  185

 185명의 유저에 대해 아래 matrix를 각각 생성 



,anime_id,Name,Genres,Type,Episodes,Source,Rating,Ranked,sypnopsis,N_votes,rating_reliability,expected_reliability,date,duration,score_by_user_4773
0,1,Cowboy Bebop,"Action,Adventure,Comedy,Drama,SciFi,Space",TV,26,Original,R - 17+ (violence & profanity),28.0,"[In, the, year, 2071,, humanity, has, colonize...",641705,0.362532,0.516247,1998.4,24.0,8.0
1,5,Cowboy Bebop: Tengoku no Tobira,"Action,Drama,Mystery,SciFi,Space",Movie,1,Original,R - 17+ (violence & profanity),159.0,"[other, day,, another, bounty—such, is, the, l...",160349,0.092112,0.106516,2001.9,115.0,8.0
2,6,Trigun,"Action,SciFi,Adventure,Comedy,Drama,Shounen",TV,26,Manga,PG-13 - Teens 13 or older,266.0,"[Vash, the, Stampede, is, the, man, with, a, $...",286146,0.165325,0.225272,1998.4,24.0,8.0
3,7,Witch Hunter Robin,"Action,Mystery,Police,Supernatural,Drama,Magic",TV,26,Original,PG-13 - Teens 13 or older,2481.0,"[ches, are, individuals, with, special, powers...",39094,0.023165,0.039668,2002.7,25.0,7.0
4,8,Bouken Ou Beet,"Adventure,Fantasy,Shounen,Supernatural",TV,52,Manga,PG - Children,3710.0,"[It, is, the, dark, century, and, the, people,...",5923,0.003675,0.007708,2004.9,23.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,44070,Tian Guan Ci Fu Special,"Historical,Supernatural,Drama",ONA,1,Novel,PG-13 - Teens 13 or older,558.0,"[Special, episode, of, Tian, Guan, Ci, Fu, .]",478,0.000301,0.006174,2021.2,22.0,6.0
4560,44087,Gintama: The Semi-Final,"Action,Comedy,Historical,Parody,Samurai,Shounen",Special,2,Manga,PG-13 - Teens 13 or older,77.0,"[New, special, that, will, tie, into, the, thi...",3727,0.002159,0.011056,2021.1,24.0,7.0
4561,44236,Senyoku no Sigrdrifa: Dai-909 Senjutsu Hime-ta...,"Action,Military",Special,1,Original,PG-13 - Teens 13 or older,8634.0,"[cap, of, the, first, eight, episodes, of, Sen...",590,0.000425,0.005962,2021.1,24.0,6.0
4562,46491,Santa Company: Manatsu no Merry Christmas,Fantasy,Movie,1,Original,PG - Children,9202.0,"[Santa, Company, is, an, original, work, broug...",827,0.000447,0.005992,2021.1,30.0,6.0


In [57]:
""" 최종 사용할 데이터 수 """

print('유저 수: ', len(all_users_df))

num = 0
for i in all_users_df:
    num += len(i)
    
print('유저 1명 당 평균 평가 갯수: ', round(num/len(all_users_df),1))

유저 수:  185
유저 1명 당 평균 평가 갯수:  2047.0


## FOR XGB

In [58]:
all_users_df_XGB = []

user_id_list = list(score_df.index)

for user_id in user_id_list:
    """ 유저 별 score vector """
    score_vector = score_df.loc[user_id] 
    score_vector = score_vector.dropna() 
    score_vector = score_vector[score_vector != 0] # 0점은 평가 안한것 (여기서 많이 날라가는듯)

    anime_id_list = list(score_vector.index)
    user_score_list = list(score_vector.values)

    """ 유저가 본 영화만 추출 """
    animes_user = animes_xgb.loc[animes_xgb['anime_id'].isin(anime_id_list)] 

    """ 유저 별 매트릭스 생성 """
    score_vector_df = pd.DataFrame(score_vector)
    score_vector_df['anime__id'] = score_vector_df.index
    score_vector_df.columns = ['score_by_user_{}'.format(user_id), 'anime__id']

    user_df = pd.merge(animes_user,score_vector_df, how='inner',on='anime_id')
    user_df = user_df.iloc[: , :-1]
    user_df = user_df.set_index('anime_id')

    """ append user_df """
    all_users_df_XGB.append(user_df)

a = len(all_users_df_XGB)
print('users: ', a)
print('\n', '{0}명의 유저에 대해 아래 matrix를 각각 생성'.format(a), '\n')
all_users_df_XGB[0]

users:  185

 185명의 유저에 대해 아래 matrix를 각각 생성 



,Episodes,Ranked,N_votes,rating_reliability,expected_reliability,date,duration,x0_Movie,x0_ONA,x0_OVA,...,91,92,93,94,95,96,97,98,99,score_by_user_4773
anime_id,,,,,,,,,,,,,,,,,,,,,
1,26,28.0,641705,0.362532,0.516247,1998.4,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.219417,0.000000,0.000000,8.0
5,1,159.0,160349,0.092112,0.106516,2001.9,115.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,8.0
6,26,266.0,286146,0.165325,0.225272,1998.4,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,8.0
7,26,2481.0,39094,0.023165,0.039668,2002.7,25.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294035,0.0,0.250298,0.000000,0.000000,7.0
8,52,3710.0,5923,0.003675,0.007708,2004.9,23.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.228532,0.438215,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44070,1,558.0,478,0.000301,0.006174,2021.2,22.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,6.0
44087,2,77.0,3727,0.002159,0.011056,2021.1,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,7.0
44236,1,8634.0,590,0.000425,0.005962,2021.1,24.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,6.0


In [59]:
""" 최종 사용할 데이터 수 """

print('유저 수: ', len(all_users_df_XGB))

num = 0
for i in all_users_df_XGB:
    num += len(i)
    
print('유저 1명 당 평균 평가 갯수: ', round(num/len(all_users_df_XGB),1))

유저 수:  185
유저 1명 당 평균 평가 갯수:  2047.0


In [60]:
import pickle

In [61]:
with open('animes.pickle', 'wb') as outfile:
    pickle.dump(animes, outfile)

In [62]:
with open('data.pickle', 'wb') as outfile:
    pickle.dump(all_users_df, outfile)

In [63]:
with open('data_xgb.pickle', 'wb') as outfile:
    pickle.dump(all_users_df_XGB, outfile)

In [64]:
with open('sypnopsis_matrix.pickle', 'wb') as outfile:
    pickle.dump(sypnopsis_matrix, outfile)

In [65]:
with open('frequency_matrix.pickle', 'wb') as outfile:
    pickle.dump(frequency_matrix, outfile)